In [1]:
from openai import OpenAI
import gradio as gr
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv(override=True)
opneai_api_key = os.getenv('OPENAI_API_KEY')
api_key = os.getenv('OR_API_KEY')



In [ ]:
MODEL = 'openai/gpt-oss-20b:free'
openai = OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=api_key
)


In [4]:
MODEL = "deepseek-r1:1.5b"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [4]:
system_message = """You are a helpful assistant for an Airline called FlightAI. 
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so."""

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


guvhjbk/
guvhjbk/
guvhjbk/


In [5]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [7]:
get_ticket_price("paris")

Tool get_ticket_price called for paris


'$899'

In [8]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]

In [16]:
openai = OpenAI(api_key=opneai_api_key)
MODEL = "gpt-4o-mini"

In [17]:
import json
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)
    city = args["destination_city"]
    price = get_ticket_price(city)

    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps({
            "destination_city": city,
            "price": price
        })
    }

In [32]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    choice = response.choices[0]

    if choice.finish_reason == "tool_calls":
        print(choice.message)
        tool_message = choice.message
        result = handle_tool_call(tool_message)
        print("DEBUG:", result, type(result))

        tool_response = handle_tool_call(tool_message)
        
        messages.append(tool_message)
        messages.append(tool_response)

        response = openai.chat.completions.create(model=MODEL, messages=messages)
        return response.choices[0].message.content
    
    return choice.message.content


In [33]:
gr.ChatInterface(fn=chat, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_0ClYtIHyYxuc9oesO0OtVpG5', function=Function(arguments='{"destination_city":"Paris"}', name='get_ticket_price'), type='function')])
Tool get_ticket_price called for Paris
DEBUG: {'role': 'tool', 'tool_call_id': 'call_0ClYtIHyYxuc9oesO0OtVpG5', 'content': '{"destination_city": "Paris", "price": "$899"}'} <class 'dict'>
Tool get_ticket_price called for Paris


In [26]:
response

''